# STA 141B Lecture 13

The class website is <https://github.com/2019-winter-ucdavis-sta141b/notes>

### Announcements

* Assignment 3 grades will be posted later today

### Topics

* Natural Language Processing (wrap-up)

### Datasets

### References

* [Natural Language Processing with Python][nlpp], chapters 1-3. Beware: the print version is for Python 2.
* [Applied Text Analysis with Python][atap], chapters 1, 3, 4

[PDSH]: https://jakevdp.github.io/PythonDataScienceHandbook/
[ProGit]: https://git-scm.com/book/
[nlpp]: https://www.nltk.org/book/
[atap]: https://search.library.ucdavis.edu/primo-explore/fulldisplay?docid=01UCD_ALMA51320822340003126&context=L&vid=01UCD_V1&search_scope=everything_scope&tab=default_tab&lang=en_US

In [2]:
import numpy as np
import pandas as pd

import nltk
import nltk.corpus

# nltk.download("gutenberg")
# nltk.download("punkt")

## Feature Engineering for Natural Language Data

Most statistical techniques take numbers as input. You may have already noticed this when working with categorical data. We can't compute the mean, median, standard deviation, or z-score if the observations aren't numbers. While we can fit linear models, it takes extra work because we have to create, or _engineer_, indicator variables.

We face the same problem with natural language data. We need to _quantify_ documents, or turn them into numbers, so that we can use a wider variety of statistical techniques. We can do this by engineering features from our documents.

So: what kinds of features can we create for language data?

### Term Frequencies

One solution is to extend the idea of frequency analysis. We used frequency analysis to study individual documents, but what if we compute the word frequencies for every document in our corpus, and use those frequencies as features?

Let's try this for a small corpus:

In [3]:
corpus = ["The cat saw the dog was angry.", "The dog saw the cat was angry.", "The canary saw the iguana was sad."]

#doc = corpus[0]
def get_freq_dist(doc):
    words = (w.lower() for w in nltk.word_tokenize(doc))
    words = (w for w in words if w not in ["the", "a"] and w.isalnum())
    return nltk.FreqDist(words)

df = pd.DataFrame([get_freq_dist(doc) for doc in corpus])
df = df.fillna(0)
df

,angry,canary,cat,dog,iguana,sad,saw,was
0,1.0,0.0,1.0,1.0,0.0,0.0,1,1
1,1.0,0.0,1.0,1.0,0.0,0.0,1,1
2,0.0,1.0,0.0,0.0,1.0,1.0,1,1


In [4]:
".".isalnum()

False

Notice that when we use term frequencies as features, we lose information about the order of the words in each document.

The first and second document contain the same words, but in different orders. The word frequency features for these two documents are identical.

The __scikit-learn__ package (included with Anaconda) provides functions to help with feature engineering. The `sklearn.feature_extraction.text` submodule is specifically for extracting features from text documents.

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer(tokenizer = nltk.word_tokenize)
freq = vec.fit_transform(corpus)

# Convert sparse matrix to dense matrix
# Don't do this for a really large matrix
freq.todense()

matrix([[1, 1, 0, 1, 1, 0, 0, 1, 2, 1],
        [1, 1, 0, 1, 1, 0, 0, 1, 2, 1],
        [1, 0, 1, 0, 0, 1, 1, 1, 2, 1]], dtype=int64)

<span style="color: #F00; font-weight: bold">NOTE</span>

_In lecture I couldn't remember the name of the method for this, but here it is now. -Nick_

Use the `.get_feature_names()` method to see which term each column corresponds to:

In [14]:
vec.get_feature_names()

['.', 'angry', 'canary', 'cat', 'dog', 'iguana', 'sad', 'saw', 'the', 'was']

In [6]:
vec

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function word_tokenize at 0x7efcf08d86a8>,
        vocabulary=None)

One problem with term frequencies is that some terms have high frequencies simply because they appear frequently in the language. These terms can cause documents to appear similar even if they are otherwise different.

While removing stopwords takes care of some high-frequency words, there may also be high-frequency words that have meaning and need to be kept.

### One-hot Encoding

We can avoid emphasis on high-frequency words by ignoring frequency altogether. Instead, we can create indicator variables for individual words. The indicator is 1 if the word appears in the document, and 0 otherwise.

In machine learning, an indicator variable is also called a _one-hot encoding_.

The `sklearn.preprocessing` submodule of __scikit-learn__ provides a function for one-hot encoding.

In [7]:
#(freq > 0).todense()

from sklearn.preprocessing import Binarizer

binarizer = Binarizer()
ohot = binarizer.fit_transform(freq)


ohot.todense()

matrix([[1, 1, 0, 1, 1, 0, 0, 1, 1, 1],
        [1, 1, 0, 1, 1, 0, 0, 1, 1, 1],
        [1, 0, 1, 0, 0, 1, 1, 1, 1, 1]], dtype=int64)

As with term frequencies, we lose information about the order of the words in the document.

One-hot encoding as an extreme transformation: every term is equally important. This means terms that are relatively rare or unique still might be underemphasized (this is also a problem for term frequencies).

### Term Frequency-Inverse Document Frequency

_Term frequency-inverse document frequency_ (tf-idf) statistics put terms on approximately the same scale while also emphasizing relatively rare terms. There are [several different tf-idf statistics](https://en.wikipedia.org/wiki/Tf%E2%80%93idf).

The _smoothed tf-idf_, for a term $t$ and document $d$, is given by:

$$
\operatorname{tf-idf}(t, d) = \operatorname{tf}(t, d) \cdot \log \left( \frac{N}{1 + n_t} \right)
$$

where $N$ is the total number of documents and $n_t$ is the number of documents that contain $t$.

The `sklearn.feature_extraction.text` submodule of __scikit-learn__ provides a function for computing tf-idf:

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer(tokenizer = nltk.word_tokenize, sublinear_tf = True)
tfidf = vec.fit_transform(corpus)

tfidf.todense()

matrix([[0.30371264, 0.39108532, 0.        , 0.39108532, 0.39108532,
         0.        , 0.        , 0.30371264, 0.5142302 , 0.30371264],
        [0.30371264, 0.39108532, 0.        , 0.39108532, 0.39108532,
         0.        , 0.        , 0.30371264, 0.5142302 , 0.30371264],
        [0.26291231, 0.        , 0.44514923, 0.        , 0.        ,
         0.44514923, 0.44514923, 0.26291231, 0.44514923, 0.26291231]])

In long documents or documents with many high-frequency terms, we can further reduce the emphasis on these terms by taking the logarithm of the term frequency. To do this, set `sublinear_tf = True` in the `TfidfVectorizer()` function.

## The Bag-of-words Model

The one-hot encoding, term frequencies, and TF-IDF scores all ignore word order.

The _bag-of-words model_ assumes that the order of words in a document doesn't matter. Imagine taking the words in each document and dumping them into a bag, where they get all mixed up. Note that in this case "model" means a way of thinking about a problem, not a statistical model.

While the order of words in a document might seem important, the bag-of-words model is surprisingly useful. The bag-of-words model is a good place to start if you want to use statistical methods on language data.

## Measuring Similarity

We can measure the _similarity_ of two documents by computing the distance between their term frequency vectors. There are many different ways we can measure distance and similarity:

* Minkowski distance, a family of distances that includes Euclidean distance and Manhattan distance.
* Mahalanobis distance, the Euclidean distance between z-scores.
* Cosine similarity, the cosine of the angle between two vectors. See [here](https://stats.stackexchange.com/a/235676/29695) for an explanation of how cosine similarity is related to correlation. Note that the range of cosine is $[-1, 1]$ and $\cos(0) = 1$, so vectors that are close together will have a cosine similarity close to 1, not 0.
* And others...

Cosine similarity often works well for language data. The cosine similarity between two vectors $a$ and $b$ is defined as:

$$
\frac{a \cdot b}{\Vert a \Vert \Vert b \Vert}
$$

where $\Vert \cdot \Vert$ is the Euclidean norm.

The `TfidfVectorizer()` function already divides the returned tf-idf vectors by their Euclidean norms, so we can compute cosine similarity as a simple dot product:

In [9]:
(tfidf @ tfidf.T).todense()

matrix([[1.        , 1.        , 0.46845855],
        [1.        , 1.        , 0.46845855],
        [0.46845855, 0.46845855, 1.        ]])

<span style="color: #F00; font-weight: bold">NOTE</span>

_Here's an answer to a question I didn't have a good answer for in lecture. -Nick_

Part of the reason that cosine similarity is a good measure in NLP is that cosine similarity, like correlation, is not affected by the scale of the vector elements. For vectors that contain term frequencies (or functions of term frequencies), this means that the length of the original documents will not affect whether or not they are similar -- only their word content will.

## The n-gram Model

Remember how the bag-of-words model ignores word order?

We can extend the model to keep some order by taking sequences of words instead of individual words. Sequences of two or three words are called _bigrams_ and _trigrams_, respectively. A sequence of $n$ words is called an _n-gram_.

__nltk__ provides functions to extract n-grams:

In [10]:
words = [nltk.word_tokenize(doc) for doc in corpus]
list(nltk.ngrams(words[0], 3))

[('The', 'cat', 'saw'),
 ('cat', 'saw', 'the'),
 ('saw', 'the', 'dog'),
 ('the', 'dog', 'was'),
 ('dog', 'was', 'angry'),
 ('was', 'angry', '.')]

In [11]:
list(nltk.ngrams(words[1], 3))

[('The', 'dog', 'saw'),
 ('dog', 'saw', 'the'),
 ('saw', 'the', 'cat'),
 ('the', 'cat', 'was'),
 ('cat', 'was', 'angry'),
 ('was', 'angry', '.')]

Notice that a separate n-gram was computed for each word in the original document. So for the bigrams in the example, we get every pair of words that appears in the document.

The n-gram model assumes that nearby words have the strongest effect on the meaning of each word.

We can use n-grams to identify phrases that are particularly common in a document. We can also use the n-gram model to engineer features, the same way we used the bag-of-words model. That is, we can compute frequencies, one-hot encodings, TF-IDFs, and other features:

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Word-tokenize data ahead of time since we'll need tokens to compute bigrams
words = [nltk.word_tokenize(d.lower()) for d in corpus]

# nltk.bigrams() is a function for computing bigrams, equivalent to nltk.ngrams(DATA, 2)
vectorizer = TfidfVectorizer(tokenizer = nltk.bigrams, lowercase = False)
tfidf = vectorizer.fit_transform(words)

tfidf.todense()

matrix([[0.35221512, 0.        , 0.46312056, 0.        , 0.        ,
         0.46312056, 0.        , 0.        , 0.27352646, 0.        ,
         0.35221512, 0.35221512, 0.        , 0.35221512, 0.        ],
        [0.35221512, 0.        , 0.        , 0.46312056, 0.46312056,
         0.        , 0.        , 0.        , 0.27352646, 0.        ,
         0.35221512, 0.35221512, 0.        , 0.35221512, 0.        ],
        [0.        , 0.39687454, 0.        , 0.        , 0.        ,
         0.        , 0.39687454, 0.39687454, 0.2344005 , 0.39687454,
         0.        , 0.        , 0.39687454, 0.        , 0.39687454]])

In [18]:
# Now each column corresponds to one bigram.
vectorizer.get_feature_names()

[('angry', '.'),
 ('canary', 'saw'),
 ('cat', 'saw'),
 ('cat', 'was'),
 ('dog', 'saw'),
 ('dog', 'was'),
 ('iguana', 'was'),
 ('sad', '.'),
 ('saw', 'the'),
 ('the', 'canary'),
 ('the', 'cat'),
 ('the', 'dog'),
 ('the', 'iguana'),
 ('was', 'angry'),
 ('was', 'sad')]

In [17]:
# As before, we can see how similar each document is to the others by examining the TF-IDF statistics.
# By using bigrams, we can see the difference in the first and second document.
# Note that the first and second document are still much more similar than the first and third document.
(tfidf @ tfidf.T).todense()

matrix([[1.        , 0.5710387 , 0.06411474],
        [0.5710387 , 1.        , 0.06411474],
        [0.06411474, 0.06411474, 1.        ]])